In [1]:
import re
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from sklearn import metrics
import matplotlib.pyplot as plt
from IPython.display import IFrame
from sklearn.cluster import KMeans
import matplotlib.font_manager as fm
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Circle, Arc

#plt.style.use('ggplot')
%matplotlib inline
pd.options.display.max_columns=25

C:\Users\btq\Anaconda\envs\py35\lib\site-packages\matplotlib\__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
cap_url = "http://stats.nba.com/stats/commonallplayers?"
cap_param = {'IsOnlyCurrentSeason':"1",
                      'LeagueID': u'00',
                      'Season': u'2015-16'}
cap_resp = requests.get(cap_url, params=cap_param)
cap_df = pd.DataFrame(cap_resp.json()['resultSets'][0]['rowSet'],columns=cap_resp.json()['resultSets'][0]['headers'])
cap_df.head()

,PERSON_ID,DISPLAY_LAST_COMMA_FIRST,ROSTERSTATUS,FROM_YEAR,TO_YEAR,PLAYERCODE,TEAM_ID,TEAM_CITY,TEAM_NAME,TEAM_ABBREVIATION,TEAM_CODE,GAMES_PLAYED_FLAG
0,203112,"Acy, Quincy",1,2012,2015,quincy_acy,1610612758,Sacramento,Kings,SAC,kings,Y
1,203919,"Adams, Jordan",1,2014,2015,jordan_adams,1610612763,Memphis,Grizzlies,MEM,grizzlies,Y
2,203500,"Adams, Steven",1,2013,2015,steven_adams,1610612760,Oklahoma City,Thunder,OKC,thunder,Y
3,201167,"Afflalo, Arron",1,2007,2015,arron_afflalo,1610612752,New York,Knicks,NYK,knicks,Y
4,201582,"Ajinca, Alexis",1,2008,2015,alexis_ajinca,1610612740,New Orleans,Pelicans,NOP,pelicans,Y


In [19]:
def format_name_last_comma_first(name):
    if len(name.split())<2:
        return 'All'
    if ',' in name:
        names = name.split(',')
    else:
        names = name.split()
    return names[-1].strip()+', '+' '.join(names[0:-1]).strip()
print(format_name_last_comma_first('James, Lebron'))

Lebron, James


In [21]:
def player_lookup(name='All',season='2015-16'):
    cap_url = "http://stats.nba.com/stats/commonallplayers?"
    cap_param = {'IsOnlyCurrentSeason':"1",
                      'LeagueID': u'00',
                      'Season': season}
    cap_resp = requests.get(cap_url, params=cap_param)
    cap_df = pd.DataFrame(cap_resp.json()['resultSets'][0]['rowSet'],columns=cap_resp.json()['resultSets'][0]['headers'])
    return cap_df

In [32]:
pname = 'Blake Griffin'
cap_df = player_lookup()
pinfo = cap_df[cap_df['DISPLAY_LAST_COMMA_FIRST']==format_name_last_comma_first(pname)]
pID = pinfo['PERSON_ID']
print(pinfo.head())
print(pID[0]))

     PERSON_ID DISPLAY_LAST_COMMA_FIRST  ROSTERSTATUS FROM_YEAR TO_YEAR  \
163     201933           Griffin, Blake             1      2009    2015   

        PLAYERCODE     TEAM_ID    TEAM_CITY TEAM_NAME TEAM_ABBREVIATION  \
163  blake_griffin  1610612746  Los Angeles  Clippers               LAC   

    TEAM_CODE GAMES_PLAYED_FLAG  
163  clippers                 Y  


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
sc_params = {u'AheadBehind': u'',
 u'ClutchTime': u'',
 u'ContextFilter': u'',
 u'ContextMeasure': u'FGA',
 u'DateFrom': u'',
 u'DateTo': u'',
 u'EndPeriod': u'',
 u'EndRange': u'',
 u'GameID': u'',
 u'GameSegment': u'',
 u'LastNGames': 0,
 u'LeagueID': u'00',
 u'Location': u'',
 u'Month': 0,
 u'OpponentTeamID': 0,
 u'Outcome': u'',
 u'Period': 0,
 u'PlayerID': pID.values,
 u'PointDiff': u'',
 u'Position': u'',
 u'RangeType': u'',
 u'RookieYear': u'',
 u'Season': u'2015-16',
 u'SeasonSegment': u'',
 u'SeasonType': u'Regular Season',
 u'StartPeriod': u'',
 u'StartRange': u'',
 u'TeamID': 0,
 u'VsConference': u'',
 u'VsDivision': u''}

baseurl = 'http://stats.nba.com/stats/shotchartdetail?'
resp2 = requests.get(baseurl,params=sc_params)
print(resp2.url)
print(resp2.json().keys())
player_shot_df = pd.DataFrame(resp2.json()['resultSets'][0]['rowSet'],columns=resp2.json()['resultSets'][0]['headers'])
print(player_shot_df.shape)
player_shot_df.head(5)